In [1]:
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# Import the dataset and encode the date
df = pd.read_csv('bitcoin.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10275 entries, 0 to 10274
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Timestamp          10275 non-null  int64  
 1   Open               10275 non-null  float64
 2   High               10275 non-null  float64
 3   Low                10275 non-null  float64
 4   Close              10275 non-null  float64
 5   Volume_(BTC)       10275 non-null  float64
 6   Volume_(Currency)  10275 non-null  float64
 7   Weighted_Price     10275 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 642.3 KB


In [4]:
df.describe()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
count,1.027500e+04,10275.000000,10275.000000,10275.000000,10275.000000,10275.000000,10275.000000,10275.000000
mean,1.419714e+09,278.644326,280.306878,275.802969,277.246748,0.398345,131.050041,277.879212
std,1.788394e+06,88.453580,86.894732,91.811155,90.642619,1.096373,407.636293,89.299344
min,1.417412e+09,109.870000,109.940000,109.870000,109.940000,0.010000,1.099400,109.940000
25%,1.417566e+09,189.000000,189.130000,189.000000,189.000000,0.010000,3.770000,189.000000
50%,1.420741e+09,284.350000,289.250000,273.550000,282.410000,0.026200,3.978155,286.025000
75%,1.421267e+09,377.000000,377.000000,377.000000,377.000000,0.060000,9.825550,377.000000
max,1.421421e+09,398.000000,398.000000,398.000000,398.000000,5.200000,1965.600000,398.000000


In [5]:
df['Timestamp']

0        1417411980
1        1417412040
2        1417412100
3        1417412160
4        1417412220
            ...    
10270    1421421240
10271    1421421300
10272    1421421360
10273    1421421420
10274    1421421480
Name: Timestamp, Length: 10275, dtype: int64

In [6]:
df['date'] = pd.to_datetime(df['Timestamp'], unit='s').dt.date
group = df.groupby('date')

Real_Price = group['Weighted_Price'].mean()
Real_Price

date
2014-12-01    366.775068
2014-12-02    375.943001
2014-12-03    377.882701
2014-12-04    377.991743
2014-12-06    378.000000
2014-12-08    375.000426
2014-12-10    379.250000
2014-12-12    366.046956
2014-12-18    342.500000
2015-01-08    291.663852
2015-01-13    260.000000
2015-01-14    176.055237
2015-01-15    162.566457
2015-01-16    193.402195
Name: Weighted_Price, dtype: float64

In [7]:
len(Real_Price)

14

In [8]:
# split data
prediction_days = 30
df_train = Real_Price[:len(Real_Price)-prediction_days]
df_test = Real_Price[len(Real_Price)-prediction_days:]

# process data
training_set = df_train.values
training_set = np.reshape(training_set, (len(training_set), 1))
sc = MinMaxScaler()
training_set = sc.fit_transform(training_set)
X_train = training_set[0:len(training_set)-1]
y_train = training_set[1:len(training_set)]
X_train = np.reshape(X_train, (len(X_train), 1, 1))

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
# Initialising the RNN
regressor = Sequential()

# Adding the input layer and the LSTM layer
regressor.add(LSTM(units=4, activation='sigmoid', input_shape=(None, 1)))

# Adding the output layer
regressor.add(Dense(units=1))

# Compiling the RNNregressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.compile(optimizer='adam', loss='mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size=5, epochs=100)

NameError: name 'X_train' is not defined

In [ ]:
# Making the predictions
test_set = df_test.values
inputs = np.reshape(test_set, (len(test_set), 1))
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (len(inputs), 1, 1))
predicted_BTC_price = regressor.predict(inputs)
predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)

In [ ]:
# Visualising the results
plt.figure(figsize=(25, 15), dpi=50, facecolor='w', edgecolor='k')
ax = plt.gca()
plt.plot(test_set, color='red', label='Real BTC Value')
plt.plot(predicted_BTC_price, color='blue', label='Predicted BTC Price')
plt.title('BTC Price Prediction', fontsize=20)
df_test = df_test.reset_index()
x = df_test.index
labels = df_test['date']
plt.xticks(x, labels, rotation='vertical')
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(10)
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(10)
plt.xlabel('Time', fontsize=10)
plt.ylabel('BTC Price(USD)', fontsize=20)
plt.legend(loc=2, prop={'size': 25})
plt.show()